# PMTiles example

This notebook will give an overview of how to create and visualize PMTiles archives.

## Creating PMTiles

For this example, we'll use the same file as used in the FlatGeobuf and GeoParquet example notebooks: a 13MB file of US counties.

We'll use Tippecanoe to convert this file into tiles. If you're on MacOS, Tippecanoe can be installed with

```bash
brew install tippecanoe
```

Otherwise, you can use Conda to install:

```bash
conda install -c conda-forge tippecanoe
```

First we'll download the file to our local directory:

In [1]:
!wget https://flatgeobuf.org/test/data/UScounties.fgb

--2023-08-23 15:54:58--  https://flatgeobuf.org/test/data/UScounties.fgb
Resolving flatgeobuf.org (flatgeobuf.org)... 185.199.108.153
Connecting to flatgeobuf.org (flatgeobuf.org)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14100008 (13M) [application/octet-stream]
Saving to: ‘UScounties.fgb’

UScounties.fgb      100%[===================>]  13.45M  7.94MB/s    in 1.7s    

2023-08-23 15:55:02 (7.94 MB/s) - ‘UScounties.fgb’ saved [14100008/14100008]



Tippecanoe has many options to customize its behavior. Here we'll use the `-zg` flag to tell Tippecanoe to deduce appropriate minimum and maximum zoom levels for the dataset. The `-o counties.pmtiles` flag tells Tippecanoe to save the output with that name.

Tippecanoe also works especially well with FlatGeobuf files. When a FlatGeobuf file is used as input, Tippecanoe will reuse the spatial index stored in the FlatGeobuf file instead of creating its own.

In [3]:
!tippecanoe UScounties.fgb -o counties.pmtiles -zg

For layer 0, using name "UScountiesfgb"
detected indexed FlatGeobuf: assigning feature IDs by sequence
3221 features, 5580299 bytes of geometry, 53296 bytes of string pool
Choosing a maxzoom of -z1 for features typically 141427 feet (43107 meters) apart, and at least 33249 feet (10135 meters) apart
Choosing a maxzoom of -z7 for resolution of about 3195 feet (973 meters) within features
  99.9%  7/36/49  


Now we have a file named `counties.pmtiles` with our data:

In [5]:
!ls -lh counties.pmtiles

-rw-r--r--@ 1 kyle  staff   2.8M Aug 23 16:02 counties.pmtiles


In [6]:
import pmtiles